In [5]:
# @title 1. 환경 설정 및 라이브러리 설치 (Cell 1)
"""
이 셀에서는 실습에 필요한 라이브러리들을 설치하고 임포트합니다.
- transformers: Hugging Face 모델 및 파이프라인 사용을 위한 라이브러리
- datasets: KLUE 데이터셋 등 Hugging Face Hub의 데이터셋 로드를 위한 라이브러리
- sentencepiece: KoBART 등 일부 모델에서 사용하는 토크나이저 라이브러리
- accelerate: 모델 로딩 및 분산 처리를 도와주는 라이브러리 (특히 NLLB 모델에 유용)
- torch: PyTorch 라이브러리 (기본 백엔드)
"""
# TODO: 필요한 라이브러리를 설치하는 명령어를 작성하세요. (transformers, datasets, sentencepiece, accelerate, torch)
# !pip install ...

import datasets
from datasets import load_dataset, DatasetDict
from transformers import pipeline
import torch
import pandas as pd # 데이터 확인용

# GPU 사용 가능 여부 확인 및 설정 (Colab에서는 보통 GPU 사용 가능)
device = 0 if torch.cuda.is_available() else -1
print(f"사용 가능한 디바이스: {'GPU' if device == 0 else 'CPU'}")

사용 가능한 디바이스: GPU


In [6]:
# @title 2. 데이터셋 로드 및 준비 (Cell 2)
"""
이 셀에서는 KLUE 데이터셋의 MRC 부분을 로드합니다.
전체 데이터셋은 클 수 있으므로, 실습을 위해 일부 데이터만 선택하여 사용합니다.
'context' 컬럼이 우리가 요약할 원본 기사 내용입니다.
"""
full_dataset = load_dataset("klue", "mrc", split="train")

# 실습을 위해 데이터 일부만 선택 (예: 앞 10개)
num_samples_to_use = 10
klue_mrc_subset = full_dataset.select(range(num_samples_to_use))

print("로드된 데이터셋 정보:")
print(klue_mrc_subset)

print("\n첫 번째 데이터 예시 (context 확인):")
print(klue_mrc_subset[0]['context'])

# 데이터셋 확인을 위해 Pandas DataFrame으로 변환 (선택 사항)
df_check = pd.DataFrame(klue_mrc_subset)
print("\n데이터셋 일부 미리보기 (DataFrame):")
display(df_check.head(3)) # Colab 환경에서는 display()가 표 형태로 보여줍니다.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrc/train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

mrc/validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

로드된 데이터셋 정보:
Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 10
})

첫 번째 데이터 예시 (context 확인):
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.

데이터셋 일부 미리보기 (DataFrame):


,title,context,news_category,source,guid,is_impossible,question_type,question,answers
0,제주도 장마 시작 … 중부는 이달 말부터,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,종합,hankyung,klue-mrc-v1_train_12759,False,1,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,"{'answer_start': [478, 478], 'text': ['한 달가량',..."
1,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_06118,True,3,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,"{'answer_start': [18, 21, 1673], 'text': ['원장 ..."
2,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_14934,False,2,지능형 생산자동화 기반기술을 개발중인 스타트업은?,"{'answer_start': [1422], 'text': ['삼보테크놀로지']}"


In [7]:
# @title 3. 요약 모델 파이프라인 로드 (Cell 3)
"""
이 셀에서는 기사 요약을 위한 KoBART 기반 모델 파이프라인을 로드합니다.
모델: gogamza/kobart-summarization
파이프라인 타입: summarization
"""
summarizer = pipeline(
    task="summarization",
    model="gogamza/kobart-summarization",
    device=device
)
print("요약 파이프라인 로드 완료.")

config.json: 0.00B [00:00, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cuda:0


요약 파이프라인 로드 완료.


In [8]:
# @title 4. 기사 내용 요약 및 데이터셋에 추가 (Cell 4)
"""
이 셀에서는 로드된 요약 파이프라인을 사용하여 데이터셋의 'context' 내용을 요약합니다.
map 함수를 사용하여 데이터셋의 각 샘플에 요약 함수를 적용하고,
결과를 'summary'라는 새로운 컬럼에 저장합니다.
"""
# 요약을 수행하는 함수 정의
def summarize_context(example):
  """데이터셋의 'context'를 받아 요약 결과를 반환하는 함수"""
  summary_result = summarizer(
        example['context'],
        max_length=150,
        min_length=30,
        do_sample=False  # deterministic 출력을 원하면 False, 다양성을 원하면 True
    )
    # 결과에서 요약 텍스트 추출
  example['summary'] = summary_result[0]['summary_text']
  return example

print("요약 작업을 시작합니다... (데이터 양에 따라 시간이 소요될 수 있습니다)")
summarized_dataset = klue_mrc_subset.map(summarize_context)
print("요약 작업 완료.")

print("\n요약이 추가된 데이터셋 정보:")
print(summarized_dataset)

print("\n첫 번째 데이터의 원문(context)과 요약(summary):")
print("--- 원문 (Context) ---")
print(summarized_dataset[0]['context'])
print("\n--- 요약 (Summary) ---")
print(summarized_dataset[0]['summary'])

# 데이터셋 확인 (Pandas)
df_check_summary = pd.DataFrame(summarized_dataset)
print("\n요약 추가 후 데이터셋 미리보기 (DataFrame):")
display(df_check_summary.head(3))

요약 작업을 시작합니다... (데이터 양에 따라 시간이 소요될 수 있습니다)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


요약 작업 완료.

요약이 추가된 데이터셋 정보:
Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers', 'summary'],
    num_rows: 10
})

첫 번째 데이터의 원문(context)과 요약(summary):
--- 원문 (Context) ---
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.

--- 요약 (Summary

,title,context,news_category,source,guid,is_impossible,question_type,question,answers,summary
0,제주도 장마 시작 … 중부는 이달 말부터,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,종합,hankyung,klue-mrc-v1_train_12759,False,1,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,"{'answer_start': [478, 478], 'text': ['한 달가량',...",올여름 장마가 17일 제주도에서 시작되었으며 서울 등 중부지방은 예년보다 사나흘 정...
1,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_06118,True,3,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,"{'answer_start': [18, 21, 1673], 'text': ['원장 ...",‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 부산의 4개 과제가 ...
2,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_14934,False,2,지능형 생산자동화 기반기술을 개발중인 스타트업은?,"{'answer_start': [1422], 'text': ['삼보테크놀로지']}",‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 부산의 4개 과제가 ...


In [9]:
# @title 5. 번역 모델 파이프라인 로드 (Cell 5)
"""
이 셀에서는 한국어 요약본을 영어로 번역하기 위한 NLLB 모델 파이프라인을 로드합니다.
모델: facebook/nllb-200-distilled-600M
파이프라인 타입: translation
NLLB 모델은 다양한 언어를 지원하며, 언어 코드를 지정해야 합니다.
한국어: kor_Hang, 영어: eng_Latn
"""
translator = pipeline(
    task="translation",
    model="facebook/nllb-200-distilled-600M",
    device=device
)

print("번역 파이프라인 로드 완료.")

# 번역 테스트 (선택 사항)
test_translation = translator(
    "안녕하세요?",
    src_lang="kor_Hang",
    tgt_lang="eng_Latn"
)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


번역 파이프라인 로드 완료.


In [10]:
# @title 6. 기사 요약본 영어로 번역 및 데이터셋에 추가 (Cell 6)
"""
이 셀에서는 생성된 'summary' 컬럼의 한국어 텍스트를 영어로 번역합니다.
map 함수를 사용하여 번역 함수를 적용하고,
결과를 'english_summary'라는 새로운 컬럼에 저장합니다.
"""
# 번역을 수행하는 함수 정의
def translate_summary_to_english(example):
  """데이터셋의 'summary'를 받아 영어 번역 결과를 반환하는 함수"""
  translation_result = translator(
        example['summary'],
        src_lang="kor_Hang",
        tgt_lang="eng_Latn",
        max_length=150,
        min_length=30,
        do_sample=False  # deterministic 출력을 원하면 False, 다양성을 원하면 True
    )
    # 결과에서 번역 텍스트 추출
  example['english_summary'] = translation_result[0]['translation_text']
  return example

print("번역 작업을 시작합니다... (모델 크기와 데이터 양에 따라 시간이 많이 소요될 수 있습니다)")
translated_dataset = summarized_dataset.map(translate_summary_to_english)
print("번역 작업 완료.")

print("\n번역이 추가된 데이터셋 정보:")
print(translated_dataset)

print("\n첫 번째 데이터의 한국어 요약(summary)과 영어 번역(english_summary):")
print("--- 한국어 요약 (Summary) ---")
print(translated_dataset[0]['summary'])
print("\n--- 영어 번역 (English Summary) ---")
print(translated_dataset[0]['english_summary'])

# 데이터셋 확인 (Pandas)
df_check_translation = pd.DataFrame(translated_dataset)
print("\n번역 추가 후 데이터셋 미리보기 (DataFrame):")
display(df_check_translation.head(3))

번역 작업을 시작합니다... (모델 크기와 데이터 양에 따라 시간이 많이 소요될 수 있습니다)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Your input_length: 136 is bigger than 0.9 * max_length: 150. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 136 is bigger than 0.9 * max_length: 150. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


번역 작업 완료.

번역이 추가된 데이터셋 정보:
Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers', 'summary', 'english_summary'],
    num_rows: 10
})

첫 번째 데이터의 한국어 요약(summary)과 영어 번역(english_summary):
--- 한국어 요약 (Summary) ---
올여름 장마가 17일 제주도에서 시작되었으며 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.

--- 영어 번역 (English Summary) ---
All-summer festivals began on the 17th in Jeju, and in central regions such as Seoul, the festival is expected to begin at the end of this month, which is about three days later than last year.

번역 추가 후 데이터셋 미리보기 (DataFrame):


,title,context,news_category,source,guid,is_impossible,question_type,question,answers,summary,english_summary
0,제주도 장마 시작 … 중부는 이달 말부터,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,종합,hankyung,klue-mrc-v1_train_12759,False,1,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,"{'answer_start': [478, 478], 'text': ['한 달가량',...",올여름 장마가 17일 제주도에서 시작되었으며 서울 등 중부지방은 예년보다 사나흘 정...,All-summer festivals began on the 17th in Jeju...
1,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_06118,True,3,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,"{'answer_start': [18, 21, 1673], 'text': ['원장 ...",‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 부산의 4개 과제가 ...,"In 2020-2021, four projects of the Ministry of..."
2,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_14934,False,2,지능형 생산자동화 기반기술을 개발중인 스타트업은?,"{'answer_start': [1422], 'text': ['삼보테크놀로지']}",‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 부산의 4개 과제가 ...,"In 2020-2021, four projects of the Ministry of..."


In [11]:
# @title 7. 감정 분석 모델 파이프라인 로드 (Cell 7)
"""
이 셀에서는 영어 텍스트의 감정을 분석하기 위한 모델 파이프라인을 로드합니다.
모델: SamLowe/roberta-base-go_emotions
파이프라인 타입: text-classification
이 모델은 다중 레이블 감정(분노, 기쁨, 슬픔 등)을 예측할 수 있습니다.
"""
emotion_classifier = pipeline(
    task="text-classification",
    model="SamLowe/roberta-base-go_emotions",
    top_k=1,
    device=device
)
print("감정 분석 파이프라인 로드 완료.")

# 감정 분석 테스트 (선택 사항)
test_emotion = emotion_classifier("I am very happy today!")
print(f"감정 분석 테스트: {test_emotion[0][0]}")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


감정 분석 파이프라인 로드 완료.
감정 분석 테스트: {'label': 'joy', 'score': 0.8936862945556641}


In [12]:
# @title 8. 영어 번역본 감정 분석 및 데이터셋에 추가 (Cell 8)
"""
이 셀에서는 번역된 'english_summary' 컬럼의 텍스트에 대해 감정 분석을 수행합니다.
map 함수를 사용하여 감정 분석 함수를 적용하고,
가장 확률이 높은 감정 레이블을 'emotion'이라는 새로운 컬럼에 저장합니다.
"""
# 감정 분석을 수행하는 함수 정의
def analyze_emotion(example):
  """데이터셋의 'english_summary'를 받아 감정 분석 결과를 반환하는 함수"""
  emotion_result = emotion_classifier(example['english_summary'])
    # 결과는 [[{'label': '...', 'score': ...}]] 형태 (top_k=1이므로 리스트 길이 1)
  example['emotion'] = emotion_result[0][0]['label']
  return example

print("감정 분석 작업을 시작합니다...")
final_dataset = translated_dataset.map(analyze_emotion)
print("감정 분석 작업 완료.")

print("\n최종 데이터셋 정보:")
print(final_dataset)

print("\n첫 번째 데이터의 영어 번역(english_summary)과 감정(emotion):")
print("--- 영어 번역 (English Summary) ---")
print(final_dataset[0]['english_summary'])
print("\n--- 분석된 감정 (Emotion) ---")
print(final_dataset[0]['emotion'])

# 최종 데이터셋 확인 (Pandas)
df_final = pd.DataFrame(final_dataset)
print("\n최종 데이터셋 미리보기 (DataFrame):")
display(df_final) # 전체 선택된 샘플 표시

감정 분석 작업을 시작합니다...


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

감정 분석 작업 완료.

최종 데이터셋 정보:
Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers', 'summary', 'english_summary', 'emotion'],
    num_rows: 10
})

첫 번째 데이터의 영어 번역(english_summary)과 감정(emotion):
--- 영어 번역 (English Summary) ---
All-summer festivals began on the 17th in Jeju, and in central regions such as Seoul, the festival is expected to begin at the end of this month, which is about three days later than last year.

--- 분석된 감정 (Emotion) ---
neutral

최종 데이터셋 미리보기 (DataFrame):


,title,context,news_category,source,guid,is_impossible,question_type,question,answers,summary,english_summary,emotion
0,제주도 장마 시작 … 중부는 이달 말부터,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,종합,hankyung,klue-mrc-v1_train_12759,False,1,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,"{'answer_start': [478, 478], 'text': ['한 달가량',...",올여름 장마가 17일 제주도에서 시작되었으며 서울 등 중부지방은 예년보다 사나흘 정...,All-summer festivals began on the 17th in Jeju...,neutral
1,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_06118,True,3,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,"{'answer_start': [18, 21, 1673], 'text': ['원장 ...",‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 부산의 4개 과제가 ...,"In 2020-2021, four projects of the Ministry of...",neutral
2,"부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,경제,acrofan,klue-mrc-v1_train_14934,False,2,지능형 생산자동화 기반기술을 개발중인 스타트업은?,"{'answer_start': [1422], 'text': ['삼보테크놀로지']}",‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 부산의 4개 과제가 ...,"In 2020-2021, four projects of the Ministry of...",neutral
3,로버트 헨리 딕,"미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에...",None,wikipedia,klue-mrc-v1_train_02900,True,3,로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?,"{'answer_start': [127], 'text': ['레이다']}","세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체...","Born in St. Louis, he earned a bachelor's degr...",neutral
4,나루세 요시히사,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,None,wikipedia,klue-mrc-v1_train_02773,False,2,개막전에서 3안타 2실점을 기록해서 패한 선수는?,"{'answer_start': [107], 'text': ['와쿠이 히데아키']}",요코하마 고등학교의 후배이기도 한 사이타마 세이부 라이온스의 와쿠이 히데아키도 개막...,"Wakui Hideaki of the Saitama Seibu Lions, who ...",neutral
5,나루세 요시히사,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,None,wikipedia,klue-mrc-v1_train_01925,True,3,개막전 선발로 등판하여 5이닝까지 노히트 처리하며 팀에 승리를 안겨준 인물은?,"{'answer_start': [107], 'text': ['와쿠이 히데아키']}",요코하마 고등학교의 후배이기도 한 사이타마 세이부 라이온스의 와쿠이 히데아키도 개막...,"Wakui Hideaki of the Saitama Seibu Lions, who ...",neutral
6,편의점 휩쓴 ‘맛집 라면’,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,생활경제,hankyung,klue-mrc-v1_train_13810,False,2,컵라면 매출에서 불닭볶음면을 이긴 상품은?,"{'answer_start': [408, 409], 'text': ['‘교동반점 짬...",유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있는 ‘검증...,The name of the famous tasting room is derived...,gratitude
7,편의점 휩쓴 ‘맛집 라면’,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,생활경제,hankyung,klue-mrc-v1_train_15862,True,3,GS25에서 판매하지 않는 컵라면은 무엇인가?,"{'answer_start': [655, 656], 'text': ['‘공화춘 짜장...",유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있는 ‘검증...,The name of the famous tasting room is derived...,gratitude
8,편의점 휩쓴 ‘맛집 라면’,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,생활경제,hankyung,klue-mrc-v1_train_08504,False,1,재작년 세븐일레븐 컵라면 중 제일 많이 팔린 제품은?,"{'answer_start': [616, 617], 'text': ['‘불닭볶음면’...",유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있는 ‘검증...,The name of the famous tasting room is derived...,gratitude
9,사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”,“앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...,기획,hankyung,klue-mrc-v1_train_13370,False,1,정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?,"{'answer_start': [661], 'text': ['환경오염이 심한 지역']}",기후변화 관련 전문 투자 사모펀드인 ACP(Asia Climate Partners)...,"Anand Prakash, of ACP (Asia Climate Partners),...",neutral


In [13]:
# @title 9. 결과 정리 및 마무리 (Cell 9)
"""
모든 단계를 거쳐 생성된 최종 데이터셋(final_dataset)에는
원본 KLUE-MRC 데이터에 'summary', 'english_summary', 'emotion' 컬럼이 추가되었습니다.
이 데이터를 CSV 파일로 저장하거나 추가 분석에 활용할 수 있습니다.
"""
print("모든 작업이 완료되었습니다.")
print("최종 데이터셋 컬럼:", final_dataset.column_names)

# 최종 결과 확인 (첫 5개 샘플)
for i in range(min(5, len(final_dataset))):
  print(f"\n--- 샘플 {i+1} ---")
  print(f"원문 일부: {final_dataset[i]['context'][:100]}...")
  print(f"요약: {final_dataset[i]['summary']}")
  print(f"영어 번역: {final_dataset[i]['english_summary']}")
  print(f"감정 분석: {final_dataset[i]['emotion']}")

모든 작업이 완료되었습니다.
최종 데이터셋 컬럼: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers', 'summary', 'english_summary', 'emotion']

--- 샘플 1 ---
원문 일부: 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전...
요약: 올여름 장마가 17일 제주도에서 시작되었으며 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.
영어 번역: All-summer festivals began on the 17th in Jeju, and in central regions such as Seoul, the festival is expected to begin at the end of this month, which is about three days later than last year.
감정 분석: neutral

--- 샘플 2 ---
원문 일부: 부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부...
요약: ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀고, 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업으로, 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다.
영어 번역: In 2020-2021, four projects of the Min

In [16]:
%ls

sample_data/


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
%ls

drive/  sample_data/


In [19]:
%cd drive/MyDrive/git

/content/drive/MyDrive/git
